In [1]:
#include <iostream>

/*a workaround to solve cling issue*/
#include "../inc/macos_cling_workaround.hpp"
/*set libtorch path, load libs*/
#include "../inc/load_libtorch.hpp"
/*import custom defined macros*/
#include "../inc/custom_def.hpp"
/*import matplotlibcpp*/
#include "../inc/load_matplotlibcpp.hpp"
/*import opencv*/
#include "../inc/load_opencv.hpp"

/*import libtorch header file*/
#include <torch/torch.h>
#include <opencv2/opencv.hpp>

# softmax回归的从零开始实现

实现softmax
$$\mathrm{softmax}(\mathbf{X})_{ij} = \frac{\exp(\mathbf{X}_{ij})}{\sum_k \exp(\mathbf{X}_{ik})}$$

In [3]:
torch::Tensor softmax(torch::Tensor X)
{
    torch::Tensor X_exp = torch::exp(X);
    auto partition = X_exp.sum(1, true);
    return X_exp / partition;
}

In [5]:
torch::Tensor X = torch::normal(0, 1, {2, 5});
torch::Tensor X_prob = softmax(X);
printT(X_prob);
printT(X_prob.sum(1));

X_prob = 
 0.4024  0.1639  0.1562  0.2227  0.0548
 0.0090  0.1674  0.3356  0.3749  0.1131
[ CPUFloatType{2,5} ]
<<--->>

X_prob.sum(1) = 
 1
 1
[ CPUFloatType{2} ]
<<--->>



实现softmax回归模型

In [15]:
torch::Tensor net(torch::Tensor X, torch::Tensor W, float b)
{   
    return softmax(torch::mm(X.reshape({-1, W.size(0)}), W) + b);
}

In [ ]:
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])